In [3]:
#import ipdb #for debugging add ipdb.set_trace()
import numpy as np
import sys
from scipy.sparse import csr_matrix
from scipy.linalg import eigh

zeors_array = np.zeros( (2, 3) )
print(zeors_array)

#Linear Shape functions and its derivatives
def GetShapeFn(z, eta, k, order):
  if(order==1):
    N_ShapeFn = np.zeros( (1, 8) )
    N_ShapeFn[0][0] = (1-z)*(1-eta)*(1-k)/8
    N_ShapeFn[0][1] = (1+z)*(1-eta)*(1-k)/8
    N_ShapeFn[0][2] = (1+z)*(1+eta)*(1-k)/8
    N_ShapeFn[0][3] = (1-z)*(1+eta)*(1-k)/8

    N_ShapeFn[0][4] = (1-z)*(1-eta)*(1+k)/8
    N_ShapeFn[0][5] = (1+z)*(1-eta)*(1+k)/8
    N_ShapeFn[0][6] = (1+z)*(1+eta)*(1+k)/8
    N_ShapeFn[0][7] = (1-z)*(1+eta)*(1+k)/8
    return N_ShapeFn

def GetShapeFnGradientwZ( z,  eta,  k, order):
  if(order==1):
    N_ShapeFnGradient = np.zeros( (1, 8) )
    N_ShapeFnGradient[0][0] = -(1-eta)*(1-k)/8
    N_ShapeFnGradient[0][1] = (1-eta)*(1-k)/8
    N_ShapeFnGradient[0][2] = (1+eta)*(1-k)/8
    N_ShapeFnGradient[0][3] = -(1+eta)*(1-k)/8
    N_ShapeFnGradient[0][4] = -(1-eta)*(1+k)/8
    N_ShapeFnGradient[0][5] = (1-eta)*(1+k)/8
    N_ShapeFnGradient[0][6] = (1+eta)*(1+k)/8
    N_ShapeFnGradient[0][7] = -(1+eta)*(1+k)/8
    return N_ShapeFnGradient

def GetShapeFnGradientwEta( z,  eta,  k,  order):
  if(order==1):
    N_ShapeFnGradient = np.zeros( (1, 8) )
    N_ShapeFnGradient[0][0] = -(1-z)*(1-k)/8
    N_ShapeFnGradient[0][1] = -(1+z)*(1-k)/8
    N_ShapeFnGradient[0][2] = (1+z)*(1-k)/8
    N_ShapeFnGradient[0][3] = (1-z)*(1-k)/8
    N_ShapeFnGradient[0][4] = -(1-z)*(1+k)/8
    N_ShapeFnGradient[0][5] = -(1+z)*(1+k)/8
    N_ShapeFnGradient[0][6] = (1+z)*(1+k)/8
    N_ShapeFnGradient[0][7] = (1-z)*(1+k)/8
    return N_ShapeFnGradient

def GetShapeFnGradientwK( z,  eta,  k,  order):
  if(order==1):
    N_ShapeFnGradient = np.zeros( (1, 8) )
    N_ShapeFnGradient[0][0] = -(1-z)*(1-eta)/8
    N_ShapeFnGradient[0][1] = -(1+z)*(1-eta)/8
    N_ShapeFnGradient[0][2] = -(1+z)*(1+eta)/8
    N_ShapeFnGradient[0][3] = -(1-z)*(1+eta)/8
    N_ShapeFnGradient[0][4] = (1-z)*(1-eta)/8
    N_ShapeFnGradient[0][5] = (1+z)*(1-eta)/8
    N_ShapeFnGradient[0][6] = (1+z)*(1+eta)/8
    N_ShapeFnGradient[0][7] = (1-z)*(1+eta)/8
    return N_ShapeFnGradient

def GetElemArr( NXElems,  NYElems,  NZElems,  NNodesPerElem):
    Elem_LocaltoGlobalNodeArr = np.zeros( (NXElems*NYElems*NZElems,NNodesPerElem) )

    count = 0
    for i in range(NZElems):#in z direction
        for j in range(NYElems):#in y direction
            for k in range(NXElems):#in x direction
                #first looping through all X dir elems then to Y dir and then to Z dir
                nElemId = k+(j)*NXElems+(i)*NXElems*NYElems
                #print('nElemId',nElemId)
                nGNodeId1 =nElemId+(j)+(i)*(NXElems+NYElems+1)
                nGNodeId2 =nElemId+(j+1)+(i)*(NXElems+NYElems+1)
                nGNodeId3 =nElemId+(j+1)+NXElems+1+(i)*(NXElems+NYElems+1)
                nGNodeId4 =nElemId+(j+1)+NXElems+(i)*(NXElems+NYElems+1)
                nGNodeId5 =nElemId+(j)+(i+1)*(NXElems+NYElems+1)+NXElems*NYElems
                nGNodeId6 =nElemId+(j+1)+(i+1)*(NXElems+NYElems+1)+NXElems*NYElems
                nGNodeId7 =nElemId+(j+1)+NXElems+1+(i+1)*(NXElems+NYElems+1)+NXElems*NYElems
                nGNodeId8 =nElemId+(j+1)+NXElems+(i+1)*(NXElems+NYElems+1)+NXElems*NYElems

                nPos=0
                Elem_LocaltoGlobalNodeArr[nElemId][nPos] = int(nGNodeId1)
                Elem_LocaltoGlobalNodeArr[nElemId][nPos+1] = int(nGNodeId2)
                Elem_LocaltoGlobalNodeArr[nElemId][nPos+2] = int(nGNodeId3)
                Elem_LocaltoGlobalNodeArr[nElemId][nPos+3] = int(nGNodeId4)
                Elem_LocaltoGlobalNodeArr[nElemId][nPos+4] = int(nGNodeId5)
                Elem_LocaltoGlobalNodeArr[nElemId][nPos+5] = int(nGNodeId6)
                Elem_LocaltoGlobalNodeArr[nElemId][nPos+6] = int(nGNodeId7)
                Elem_LocaltoGlobalNodeArr[nElemId][nPos+7] = int(nGNodeId8)
                #print('Elem_LocaltoGlobalNodeArr[nElemId]',Elem_LocaltoGlobalNodeArr[nElemId])
            
                count+= 1
    return Elem_LocaltoGlobalNodeArr

def GetGNodeCordsArr( NXElems,  NYElems,  NZElems, NNodesPerElem,  dim,  xMin,  yMin,  zMin, xMax,  yMax,  zMax):
    xLength = xMax - xMin
    yLength = yMax - yMin
    zLength = zMax - zMin
    xLe = xLength/NXElems #length of each elem in x direction
    yLe = yLength/NYElems #length of each elem in y direction
    zLe = zLength/NZElems #length of each elem in z direction
    nNodesX = NXElems+1
    nNodesY = NYElems+1
    nNodesZ = NZElems+1

    Elem_GlobalNodeCordsArr = np.zeros(( (NXElems+1)*(NYElems+1)*(NZElems+1),dim) )

    count = 0

    for i in range(NZElems+1): # in z direction
        for j in range(NYElems+1):#in y direction
            for k in range(NXElems+1):#in x direction
                nodeID = k+(j)*(NXElems+1)+(i)*(NXElems+1)*(NYElems+1)
                Elem_GlobalNodeCordsArr[nodeID][0]=(k)*xLe; 
                Elem_GlobalNodeCordsArr[nodeID][1]=(j)*yLe; 
                Elem_GlobalNodeCordsArr[nodeID][2]=(i)*zLe;   

                count=+1
    return Elem_GlobalNodeCordsArr

# Elasticity tensor
def Cijkl( i,  j,   k, l):
    E = 1000 #//Pa
    Nu = 0.3 
    lambdaVal = E/(2*(1+Nu))
    Mu = E*Nu/((1+Nu)*(1-2*Nu))

    return lambdaVal * (i==j) * (k==l) + 2 * Mu * ((i==k)*(j==l) + (i==l)* (j==k))

def GetNodalStiffness(NdashX, dim1, dim2):
    dim=3
    KAB = np.zeros( (dim,dim) )
    NdashX_Transpose = np.transpose(NdashX)

    for i in range(dim):
        for j in range(dim):
            for k in range(dim):
                for l in range(dim):
                    KAB[i][k]= KAB[i][k]+ NdashX[dim1][j]*Cijkl(i,j,k,l)*NdashX_Transpose[l][dim2]
    return KAB

def delta(i, k):
    return (i==k)

def GetNodalMass(N,rho,delta,dim1,dim2):
    dim=3
    M=np.zeros((dim,dim));
    for i in range(dim):
        for k in range(dim):
            M[i][k]=N[0][dim1]*rho*delta(i,k)*N[0][dim2]
    return M


#def GetKMandF3DFE(int NElemsX, int NElemsY, int NElemsZ, Eigen::MatrixXf GNodeCordsArr, Eigen::MatrixXd  ElemArrQuad, int order, int bondaryCondition, float F, Eigen::MatrixXf  &KGlobal, Eigen::MatrixXf  &MGlobal, Eigen::MatrixXf  &FGlobal)
def GetKMandF3DFE( NElemsX,  NElemsY,  NElemsZ, GNodeCordsArr, ElemArrQuad, order, bondaryCondition, F):
    #Inputs initialization
    E = 1000 #Pa
    Nu = 0.3 
    rho = 1
    g1 = 0 
    g2 = 0.05 # m, 

    NumNodesTotal = (NElemsX+1)*(NElemsY+1)*(NElemsZ+1)

    lambdaVal = E/(2*(1+Nu))
    Mu = E*Nu/((1+Nu)*(1-2*Nu))

    NumOfElems = NElemsX*NElemsY*NElemsZ
    NPerElems =8 #nodes per Elem
    dim = 3 #DOF

    xMin = 0.0
    yMin = 0.0
    zMin = 0.0
    xMax = 0.1
    yMax = 0.1
    ZMax = 1.0
    numElems = NElemsX*NElemsY*NElemsZ
    NumNodesPerElem = 8
    if (order==1):
        NumNodesPerElem=8  
    elif (order==2):
        NumNodesPerElem=20

    quadPts = [-0.7746,0,0.7746]   #-sqrt(3/5),0,sqrt(3/5)  
    NumQuadPts = len(quadPts)
    WeightquadPts = [0.5555,0.8888,0.5555]  #5/9,8/9,5/9  


    KGlobal = np.zeros((dim*NumNodesTotal,dim*NumNodesTotal))
    MGlobal = np.zeros((dim*NumNodesTotal,dim*NumNodesTotal))
    FGlobal = np.zeros((dim*NumNodesTotal,1))

    for i in range(NumOfElems):
        Klocal = np.zeros( (NumNodesPerElem*dim,NumNodesPerElem*dim) )
        Mlocal = np.zeros( (NumNodesPerElem*dim,NumNodesPerElem*dim) )
        Flocal = np.zeros( (NumNodesPerElem*dim,1) )

        for j in range(NumQuadPts):
            for k in range(NumQuadPts):
                for l in range(NumQuadPts):
                    #For Klocal
                    if (order == 1):
                        N = GetShapeFn(quadPts[j],quadPts[k],quadPts[l],1)
                        NwZ = GetShapeFnGradientwZ(quadPts[j],quadPts[k],quadPts[l],1)
                        NwEta = GetShapeFnGradientwEta(quadPts[j],quadPts[k],quadPts[l],1)
                        NwK = GetShapeFnGradientwK(quadPts[j],quadPts[k],quadPts[l],1)

                        nodeid1 = ElemArrQuad[i][0]
                        nodeid2 = ElemArrQuad[i][1]
                        nodeid3 = ElemArrQuad[i][2]
                        nodeid4 = ElemArrQuad[i][3]
                        nodeid5 = ElemArrQuad[i][4]
                        nodeid6 = ElemArrQuad[i][5]
                        nodeid7 = ElemArrQuad[i][6]
                        nodeid8 = ElemArrQuad[i][7]

                        nodeIds = np.array([nodeid1, nodeid2, nodeid3, nodeid4, nodeid5, nodeid6, nodeid7, nodeid8])
                      
                        elemNodeLs = [int(x) for x in ElemArrQuad[i]]
                        a = np.arange(1) #to pass 0 as arg to matrix slicing np.ix_(NodeIdls,a)
                        b = np.arange(1,2)
                        c = np.arange(2,3)
                        Jacobian = np.zeros( (dim,dim) )
                        Jacobian = np.array((np.dot(NwZ,GNodeCordsArr[np.ix_(elemNodeLs, a)]),   np.dot(NwEta,GNodeCordsArr[np.ix_(elemNodeLs, a)]),  np.dot(NwK,GNodeCordsArr[np.ix_(elemNodeLs, a)]), np.dot(NwZ,GNodeCordsArr[np.ix_(elemNodeLs, b)]), np.dot(NwEta,GNodeCordsArr[np.ix_(elemNodeLs, b)]), np.dot(NwK,GNodeCordsArr[np.ix_(elemNodeLs, b)]), np.dot(NwZ,GNodeCordsArr[np.ix_(elemNodeLs, c)]), np.dot(NwEta,GNodeCordsArr[np.ix_(elemNodeLs, c)]), np.dot(NwK,GNodeCordsArr[np.ix_(elemNodeLs, c)])))
                        Jacobian = np.reshape(Jacobian, (dim, dim))
                        invJacobian = np.linalg.inv(Jacobian)

                        NdashZEtaX = np.zeros( (NPerElems,dim) )
                        NdashZEtaX[:,0]= NwZ
                        NdashZEtaX[:,1]= NwEta
                        NdashZEtaX[:,2]= NwK
                        NdashX = np.dot(NdashZEtaX, invJacobian)
 
                        kL = np.zeros( (NumNodesPerElem*dim,NumNodesPerElem*dim) )
                        mL = np.zeros( (NumNodesPerElem*dim,NumNodesPerElem*dim) )

                        for m in range(NumNodesPerElem):
                            for n in range(NumNodesPerElem):
                                KAB = GetNodalStiffness(NdashX, m, n)
                                MAB=GetNodalMass(N,rho,delta,m,n);
                                a11 = np.arange(3*m,3*(m+1))
                                b11 = np.arange(3*n,3*(n+1))
                                kL[np.ix_(a11, b11)]=KAB
                                mL[np.ix_(a11, b11)]=MAB;
                        Klocal += kL*np.linalg.det(Jacobian)*WeightquadPts[j]*WeightquadPts[k]*WeightquadPts[l]
                        Mlocal += mL*np.linalg.det(Jacobian)*WeightquadPts[j]*WeightquadPts[k]*WeightquadPts[l]
                        Flocal += 0
                       
        for m in range(NumNodesPerElem):
            for n in range(NumNodesPerElem):
                I=np.arange(3*(int(ElemArrQuad[i][m])),3*int(ElemArrQuad[i][m])+3)
                J=np.arange(3*(int(ElemArrQuad[i][n])),3*int(ElemArrQuad[i][n])+3)
                a1 = np.arange(3*m,3*(m+1))
                b1 = np.arange(3*n,3*(n+1))
                KGlobal[np.ix_(I, J)]+= Klocal[np.ix_(a1, b1)]
                MGlobal[np.ix_(I, J)]+= Mlocal[np.ix_(a1, b1)]
    return KGlobal, MGlobal, FGlobal

def GetDGlobalArr( NElemsX,  NElemsY,  NElemsZ,  order,  dim,  bondaryCondition, F, transient):
    NumOfElems = NElemsX*NElemsY*NElemsZ
    NPerElems =8 #nodes per Elem

    xMin = 0.0
    yMin = 0.0
    zMin = 0.0
    xMax = 0.1
    yMax = 0.1
    zMax = 1.0
    g2 = 0.05 #displacement at z=zMax
    NumNodesTotal = (NElemsX+1)*(NElemsY+1)*(NElemsZ+1)

    if (order==1):
        NumNodesTotal = (NElemsX+1)*(NElemsY+1)*(NElemsZ+1) #linear
        NPerElems = 8
    
    #createmesh
    ElemArrQuad = GetElemArr(NElemsX, NElemsY, NElemsZ, NPerElems)
    #Get Node coords
    GNodeCordsArr = GetGNodeCordsArr(NElemsX, NElemsY,NElemsZ, NPerElems ,dim, xMin, yMin, zMin, xMax, yMax, zMax)
  
    #Getting KGlobal, MGlobal, FGlobal
    KGlobal, MGlobal, FGlobal = GetKMandF3DFE(NElemsX, NElemsY, NElemsZ, GNodeCordsArr, ElemArrQuad, order, bondaryCondition, F)
    
    DirichBC = []            
    j =0
    for i in range(NumNodesTotal):
        if(GNodeCordsArr[i][2]==0):   #at z=0
            new=[]
            new.append(3*i)
            new.append(0)
            DirichBC.append(new)

            new=[]
            new.append(3*i+1)
            new.append(0)
            DirichBC.append(new)

            new=[]
            new.append(3*i+2)
            new.append(0)
            DirichBC.append(new)
            j=j+3
        if(GNodeCordsArr[i][2]==zMax):   #at z=zMax
            new=[]
            new.append(3*i+1)
            new.append(g2)
            DirichBC.append(new)
            j=j+1

    DirichBC = np.array(DirichBC)
    nDirichletnodes=len(DirichBC[:,0])
    nNotDirchiletnodes = dim*NumNodesTotal-nDirichletnodes

    #Applying Dirichilet Boundary conditions
# //  //Apply Dirichlet's BCs
# //      3D Hexahedral Mesh: ([0, 0.1]x[0, 0.1]x[0, 1.0], use a 4 x 4 x 40 element mesh.)
# //      Fully constrained on z = 0m face: g = 0 along z = 0m
# //      -displacement on z = 1m face: g2 = 0.05 along z = 1.0m
  
    #KDGlobal, MDGlobal and FDGlobal Dirichlet
    KDGlobal = np.zeros((nNotDirchiletnodes,nNotDirchiletnodes))
    MDGlobal = np.zeros((nNotDirchiletnodes,nNotDirchiletnodes))
    FDGlobal = np.zeros((nNotDirchiletnodes,1))
    NotDirchNodes = np.zeros((nNotDirchiletnodes,1))
  
    #NonDirchlet Nodes matrix
    count=0
    flag=0
    for i in range(dim*NumNodesTotal):
        flag=0
        for k in range(nDirichletnodes):
            if(i==DirichBC[k][0]):
                flag=1
        if(flag==0):
            NotDirchNodes[count][0]=i
            count +=1
  
    #KDGlobal & MDGlobal
    for k in range(nNotDirchiletnodes):
        for l in range(nNotDirchiletnodes):
            KDGlobal[k][l]=KGlobal[int(NotDirchNodes[k][0])][int(NotDirchNodes[l][0])]
            MDGlobal[k][l]=MGlobal[int(NotDirchNodes[k][0])][int(NotDirchNodes[l][0])]

    #Find Eigen value and vector for the generalized eigen value
    #from scipy.linalg import eigh
    #eigvals, eigvecs = eigh(A, B, eigvals_only=False, subset_by_index=[0, 1, 2])
    import timeit
    starttime = timeit.default_timer()
    eigvals, eigvecs = eigh(KDGlobal, MDGlobal, eigvals_only=False)
    print('time taken for Eigen values calc', timeit.default_timer() - starttime)
    print('eigvals', eigvals)
    print('eigvals', eigvals.shape)
    print('eigvals', eigvecs)
    print('eigvecs', eigvecs.shape)

    #fdash
    F_forSubstract = np.zeros((nNotDirchiletnodes,1))
    for i in range(nNotDirchiletnodes):
        for j in range(nDirichletnodes):
            F_forSubstract[i][0]=KGlobal[int(NotDirchNodes[i][0])][int(DirichBC[j][0])]*DirichBC[j][1] + F_forSubstract[i][0]
 
    #FDGlobal
    for i in range(nNotDirchiletnodes):
        FDGlobal[i][0]=FGlobal[int(NotDirchNodes[i][0])][0] - F_forSubstract[i][0]


    DGlobal = np.zeros((dim*NumNodesTotal,1))

    #import time
    #start = time.time()
    if (transient==0):
        DGlobal_D = np.zeros((dim*NumNodesTotal-nDirichletnodes,1))

        DGlobal_D = np.dot(np.linalg.inv(KDGlobal), FDGlobal)

        for m in range(nDirichletnodes):
            DGlobal[int(DirichBC[m][0])][0]=DirichBC[m][1]

        for n in range(nNotDirchiletnodes):
            DGlobal[int(NotDirchNodes[n][0])][0]=DGlobal_D[n][0]
        

        NotDirchNodes = [int(x) for x in NotDirchNodes]
        print('DGlobalArrNotDirchNodes',DGlobal[NotDirchNodes,0].max())
        
        #Transient analysis
    if(transient==1): #=1 for transient 

        a=1
        b=0.001

        beta=0.255;
        gamma=2*beta;

        tDelta=1000;   
        tSteps=50; 

        u0=np.zeros((dim*NumNodesTotal,1));  #initial displacement
        v0=np.zeros((dim*NumNodesTotal,1));  #initial velocity
        
        CDGlobal = np.zeros((nNotDirchiletnodes,nNotDirchiletnodes))
        CDGlobal=(a*MDGlobal + b*KDGlobal);   #Rayleigh damping
        
        vGlobal = np.zeros((dim*NumNodesTotal,1))
        v = np.zeros((dim*NumNodesTotal-nDirichletnodes,1))
        acc = np.zeros((dim*NumNodesTotal-nDirichletnodes,1))
        DGlobal_D = np.zeros((dim*NumNodesTotal-nDirichletnodes,1))
        DGlobal_D

        #Initializing
        DGlobal[:,0]=u0[:,0];
        vGlobal[:,0]=v0[:,0];
        
        #time stepping #Newmmark
        NotDirchNodes = [int(x) for x in NotDirchNodes]
        DGlobal_D[:,0]=DGlobal[NotDirchNodes,0]
        DGlobal_D = np.array(DGlobal_D)

        v[:,0]=vGlobal[NotDirchNodes,0]
        v = np.array(v)

        acc=np.dot(np.linalg.inv(MDGlobal), (FDGlobal - np.dot(KDGlobal,DGlobal_D[:,0])[...,None] - np.dot(CDGlobal,v[:,0])[...,None] ))
        
        for i in range(tSteps):
            d_tilde= DGlobal_D[:,i][...,None] + np.dot(tDelta,v[:,i][...,None]) + ((1-2*beta)/2*(tDelta**2))*acc[:,i][...,None]
            v_tilde= v[:,i][...,None] + (1-gamma)*tDelta*acc[:,i][...,None]
            
            accToAppend = np.dot(np.linalg.inv(MDGlobal+(gamma*tDelta*CDGlobal)+(beta*(tDelta**2)*KDGlobal)) ,(FDGlobal - np.dot(KDGlobal,d_tilde) - np.dot(CDGlobal,v_tilde)))
            acc = np.append(acc, accToAppend, 1)

            DToAppend = d_tilde + np.dot(beta*(tDelta**2),acc[:,i+1])[...,None]
            DGlobal_D = np.append(DGlobal_D, DToAppend, 1)

            v = np.append(v, v_tilde + gamma*tDelta*acc[:,i+1][...,None], 1)
        
        #assigning values from last time step to DGlobal
        DGlobal = np.array(DGlobal)
        DirichBCIndx = [int(x) for x in DirichBC[:,0]]
        DGlobal[DirichBCIndx]=DirichBC[:,1][...,None];

        for i in range(tSteps):
            DGlobal[NotDirchNodes]=DGlobal_D[:,i][...,None];
            #print('DGlobal_D',DGlobal_D)
            #print('DGlobal_D',DGlobal_D.shape)
            filename = 'HW4DGlobalArr4440Elems'+str(i)+'.txt'
            np.savetxt(filename, DGlobal, fmt='%1.7f')
        print("DGlobalDone\n")
    return ElemArrQuad, GNodeCordsArr, DGlobal

def main():
    #Inputs
    E = 1000 #//Pa
    Nu = 0.3 #//Pa
    g1 = 0
    g2 = 0.05 # m, 

    NElemsX =4 #horizontal
    NElemsY =4 # vertical
    NElemsZ =40 #/ Z direction
    NElemsX =2 #horizontal
    NElemsY =2 # vertical
    NElemsZ =2 # Z direction
    NumOfElems = NElemsX*NElemsY*NElemsZ
    NumNodesTotal = (NElemsX+1)*(NElemsY+1)*(NElemsZ+1)
    NPerElems =8
    dim = 3
    xMin = 0.0
    yMin = 0.0
    zMin = 0.0
    xMax = 0.1
    yMax = 0.1
    zMax = 1.0


    lambdaVal = E/(2*(1+Nu))
    Mu = E*Nu/((1+Nu)*(1-2*Nu))

    order = 1
    transient=0 #0-Static 1-Dynamic

    F=0.0
    bondaryCondition = 1

    #Getting DGlobal
    print("DGlobal Start\n")
    ElemArrQuad, GNodeCordsArr, DGlobalArr = GetDGlobalArr(NElemsX, NElemsY,NElemsZ, order, dim, bondaryCondition, F, transient)
    np.savetxt('HW4DGlobalArr4440Elems.txt', DGlobalArr, fmt='%1.7f')
    print("DGlobal Done\n")
    print("write data to the file\n")

    #Write mesh array for output
    ElemArrQuadForPrint = np.zeros( (NElemsX*NElemsY*NElemsZ,NPerElems) )
    for i in range(NumOfElems):
        for x in range(NPerElems):
          ElemArrQuadForPrint[i][x] = ElemArrQuad[i][x]+1
    np.savetxt('HW4ElemArr4440Elems.txt', ElemArrQuadForPrint,fmt='%i')
    
    #Write node coordinates array 
    np.savetxt('HW4NodeCoordsArr4440Elems.txt', GNodeCordsArr,fmt='%1.2f')
    print("Completed writing data to the file\n")
    
    return 0

main()


[[0. 0. 0.]
 [0. 0. 0.]]
DGlobal Start

time taken for Eigen values calc 0.005358475999997836
eigvals [3.04991082e+02 6.83005639e+03 8.11919837e+03 1.37572558e+04
 1.40937715e+04 8.40237871e+04 1.39106458e+06 1.39324738e+06
 1.40893861e+06 1.47240802e+06 1.47631983e+06 2.61090485e+06
 2.61092118e+06 2.67803183e+06 2.77433306e+06 2.80647546e+06
 2.94746101e+06 3.22271687e+06 3.69372532e+06 4.60732765e+06
 4.62827558e+06 5.54485920e+06 5.54878103e+06 5.57436993e+06
 5.63049223e+06 5.64083963e+06 6.92492061e+06 6.93964441e+06
 6.97997762e+06 6.98063860e+06 7.53875765e+06 7.53958741e+06
 8.76113224e+06 1.11051380e+07 1.11481992e+07 1.29367356e+07
 1.29463586e+07 1.29643112e+07 1.30209872e+07 1.30240936e+07
 1.43340827e+07 1.57065328e+07 1.84651250e+07 1.84754643e+07
 1.84982181e+07]
eigvals (45,)
eigvals [[ 6.56637320e+00 -1.35499169e-01 -1.30757713e-02 ... -3.21090768e+01
   4.63673682e-03 -4.53371299e+01]
 [ 1.21381111e-02 -1.40127414e-01  1.72881076e+01 ...  5.20107677e-03
  -5.19667270

0